In [56]:
# Ensure tensorflow is available in the notebook kernel
%pip install tensorflow -q

import os
# import pickle
# import random
# import imageio
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
# import matplotlib.cm as cm
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score, roc_auc_score, classification_report
# import tensorflow as tf
# from tensorflow import keras
from keras.utils import Sequence, to_categorical
from keras.optimizers import Adam
# from keras import optimizers, metrics, layers, models, applications
# from keras.callbacks import ReduceLROnPlateau
# from keras.models import load_model, Model, Sequential
# # from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, GlobalAveragePooling2D, PReLU, Dropout, BatchNormalization
# from keras.applications import ResNet50

Note: you may need to restart the kernel to use updated packages.


In [53]:
# in this section, i have iterated through all the images in the covid and normal folder
# the images have been labelled if they are covid or normal, covid = 1 and normal = 0
# converted those images to RGB, then resize them and make them consistent size 
# turned those images into numpy arrays and printed their shape 
folder_path = r'C:\_Dhruti\Projects\data-visualisation-learning\data\raw'
types = ['covid', 'normal']

data = [] 
labels = []

for t in types:
    path = os.path.join(folder_path, t)
    label = 1 if t == 'covid' else 0
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        # print (img_path) 
        img = Image.open(img_path).convert('RGB')           # ensures 3 channels because not everytime there wil be RGB 
        img = img.resize((224, 224))                        # resizing to make it consistent 
        img_array = np.array(img) / 255.0                   # converting to array and normalizing
        data.append(img_array)                              # saving the image data
        labels.append(label)                                # saving the label 

data = np.array(data)
labels = np.array(labels)
print (f"Data shape: {data.shape},\n Labels shape: {labels.shape}")

Data shape: (94, 224, 224, 3),
 Labels shape: (94,)


In [ ]:
# in this section, i have broken the data into 2 sets - train set and test set using
# the train_test_split function from sklearn
# and then convert the labels to categorise into 2 classes - covid and normal using to_categorical function from keras 
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=0.2, random_state=42, stratify=labels)  # previous line was to split the dataset into train and test sets 
print("training data shape: ", X_train.shape)
print("test data shape: ", X_test.shape)

y_train_category = to_categorical(Y_train, num_classes=2)
y_test_category = to_categorical(Y_test, num_classes=2)

print(y_train_category[:5])

training data shape:  (75, 224, 224, 3)
test data shape:  (19, 224, 224, 3)
[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [59]:
model = Sequential([
    # First convolutional block
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D((2,2)),
    
    # Second convolutional block
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    
    # Third convolutional block
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    
    # Flatten and fully connected layers
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # prevents overfitting
    Dense(1, activation='sigmoid')  # binary classification
])

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',   # for 2-class problems
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,           # training data
    validation_data=(X_test, y_test),
    epochs=10,                  # start small; increase later
    batch_size=16               # adjust based on memory
)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")


NameError: name 'y_train' is not defined